# Making pairwise alignments with clustalo

In [1]:
import os
import glob
import Bio
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio import AlignIO

In [2]:
 os.chdir("/home/julybel/py_proj/families/newfiles4/") # newfiles4 - version of working directory
!pwd

/home/julybel/py_proj/families/newfiles4


##### Here we prepare our sample for Clustal

In [3]:
 !ls
os.chdir("./clustalo_v7/") # version of working directory

70v7_train.zip	      ncRNApair_datav7_train.npy   RF02981.fa
clusalo		      ncRNApair_labelv5.npy	   RF03015.fa
clustalo_v7	      ncRNApair_labelv6.npy	   RF03112.fa
fam70_test.txt	      ncRNApair_labelv7_train.npy  RF03115.fa
fam70_train.txt       RF00160.fa		   sample_2.zip
fam.txt		      RF00247.fa		   sample.fasta
genelabel_sample.txt  RF00677.fa		   sample_first_70.fasta
HOWMUCHFAM.txt	      RF01758.fa		   sample_second.fasta
ncRNApair_datav5.npy  RF02115.fa		   sample_v6_100.zip
ncRNApair_datav6.npy  RF02272.fa		   train70_pairwise.zip


In [4]:
# Here you can check how much sequences contain your sample-file
dictt = SeqIO.to_dict(SeqIO.parse("sample_second.fasta", "fasta"))
fname_list = list(dictt.keys())
len(fname_list)

62

##### Now we are going to make a lot of PAIRWISE-ALIGNMENT files

In [6]:
i = 0
# !mkdir clustalo
seq1, seq2 = fname_list[0], fname_list[2]


for num1 in range(i, len(fname_list)): #(i, len(fname_list)):     (len(fname_list))
    for num2 in range(i + 1, len(fname_list)): #(i + 1, len(fname_list)):    (len(fname_list))
        seq1, seq2 = fname_list[num1], fname_list[num2]
        
        out = open('./depo.fasta', 'w')
        out.write(">" + str(dictt[seq1].id) + '\n' + str(dictt[seq1].seq) + '\n' + ">" + str(dictt[seq2].id) + '\n' + str(dictt[seq2].seq) + '\n')
        out.close()
        ! clustalo -i ./depo.fasta -o ./'{seq1}_{seq2}'.fasta
    i += 1

In [28]:
# To check how much pairwise-alignment files in your working directory
sum(list(range(62)))

1891

In [7]:
! rm depo.fasta

# Making matrices

##### Firstly, we'll check, if there're the number of files we need in our 'clustalo' directory

In [26]:
len(fname_list)**2 == len(glob.glob('*.fasta'))

False

In [7]:
#os.mkdir('./matrices')
os.chdir('./matrices/')
os.getcwd()

'/home/julybel/py_proj/families/newfiles/clustalo/matrices'

##### Making matrices

In [8]:
'''
This function returns you one matrix with shape == (1, 1200, 10)
'''
def make_matrix(fname):
    # input file with pairwise alignment
    alignment = AlignIO.read(fname, "fasta")
    out_name = str(fname)[:-6]
    
    # making file for matrix
    out = open('mat_' + str(out_name) + '.txt', 'a')
    for record in alignment:
        out.write(str(record.seq) + '\n')
        alignment_len = len(str(record.seq).strip())
    out.close()    
    
    # making dataset 2 x alignment_len
    fname = open('mat_' + str(out_name) + '.txt', 'r')
    #print(fname)
    seq1 = list(fname.readline().strip())
    seq2 = list(fname.readline().strip())
    
    # print(len(seq1), len(seq2))
    
    
    fname.close()
    data = pd.DataFrame(np.array((seq1, seq2)))
    
    # making ZERO-matrix and dataset 10 x alignment_len
    matrix = np.zeros((10, 1200))  #((10, 1200)) # or ((10, 1200))
    mat_data = pd.DataFrame(matrix)
    
    # [A1 T1 G1 C1 GAP1][A2 T2 G2 C2 GAP2]
    # [00 01 02 03 0004][05 06 07 08 0009]
    # print(alignment_len)
    for num in range(alignment_len):
        sym1 = data[num][0]
        sym2 = data[num][1]
        
        if sym1: #== 'A' or sym1 == 'T' or sym1 == 'G' or sym1 == 'C' or sym1 == '-':
            if sym1 == 'A': mat_data[num][0] = 1
            elif sym1 == 'T': mat_data[num][1] = 1
            else:
                if sym1 == 'G': mat_data[num][2] = 1
                elif sym1 == 'C': mat_data[num][3] = 1
                else: mat_data[num][4] = 1
            
        if sym2: #== 'A' or sym2 == 'T' or sym2 == 'G' or sym2 == 'C' or sym2 == '-':
            if sym2 == 'A': mat_data[num][5] = 1
            elif sym2 == 'T': mat_data[num][6] = 1
            else:
                if sym2 == 'G': mat_data[num][7] = 1
                elif sym2 == 'C': mat_data[num][8] = 1
                else: mat_data[num][9] = 1    
    matrix1 = mat_data.astype(int)
    # print(matrix1)
    
    # Matrix reshaping
    matrix = np.swapaxes(matrix, 1, 0).reshape(1, 1200, 10)
    return matrix
    
    # Saving binary file in .npy format
     #print(matrix.shape)
    
    os.remove('mat_' + str(out_name) + '.txt')
    #np.save(str(out_name) + '.npy', matrix)

##### OLD VARIANT; SEE NEW VARIANT BELOW

In [19]:
 MakeMatrix = make_matrix('./')

In [8]:
'''
I check if our sequences have the same length, don't look at this cell
'''
for file in glob.glob("mat*.txt"):
    with open(file, 'r') as fname:
        seq1 = fname.readline().strip()
        seq2 = fname.readline().strip()
        if (len(seq1) == len(seq2)) != True:
            print(fname)

In [1]:
# !ls

In [11]:
# We shoul copy all files with pairwise alignments into the our working directory
mat = int(0)
sp = []
for file in glob.glob("*.fasta"):
    print(file)
    sp.append(file)
    if type(mat) == int:
        mat = make_matrix(file)
    else:
        nwm = make_matrix(file)
        mat = np.concatenate((mat, nwm), axis=0)
mat.shape
print(len(sp))
# mat_RF02272_JH835498.1,4_RF03115_URS0000D65EA6_682634,5.txt

RF02981_URS0000D6C41F_12908,1_RF00677_JNGA01013975.1,9.fasta
RF00247_CM003506.1,3_RF00247_FAOM01002169.1,3.fasta
RF02115_AYUG01149701.1,2_RF03015_URS0000D6AE16_12908,8.fasta
RF03112_URS0000D6B04D_12908,6_RF00677_CM001221.2,9.fasta
RF02115_AAPE02007756.1,2_RF02115_FR853097.2,2.fasta
RF02981_URS0000D67A8F_12908,1_RF02115_CM000554.1,2.fasta
RF03015_URS0000D6CE4F_12908,8_RF00677_APLE01009870.1,9.fasta
RF00247_GL377605.1,3_RF00160_CM000843.2,7.fasta
RF02981_URS0000D66007_12908,1_RF02115_CM000666.2,2.fasta
RF01758_CP007039.1,0_RF02981_URS0000D66FE6_12908,1.fasta
RF00247_AOTI010773815.1,3_RF03015_URS0000D65F98_12908,8.fasta
RF01758_MUIN01000009.1,0_RF02272_CM000299.1,4.fasta
RF02981_URS0000D67944_12908,1_RF00247_JRRC01446979.1,3.fasta
RF02981_URS0000D69ECC_12908,1_RF03015_URS0000D67C15_12908,8.fasta
RF02981_URS0000D67944_12908,1_RF00677_CM000834.2,9.fasta
RF00247_CM001748.1,3_RF03112_URS0000D68791_12908,6.fasta
RF00247_FAOM01002169.1,3_RF02272_ALWT01213357.1,4.fasta
RF01758_JXDG01000018.1,0_R

RF01758_CP007039.1,0_RF02981_URS0000D68152_12908,1.fasta
RF03015_CP017717.1,8_RF00677_CM000834.2,9.fasta
RF02981_URS0000D66007_12908,1_RF00160_AOTI010592543.1,7.fasta
RF03115_URS0000D6959E_561230,5_RF00160_BDDD01000122.1,7.fasta
RF01758_AMWJ01000044.1,0_RF03015_URS0000D6AE16_12908,8.fasta
RF03115_URS0000D68E9D_104623,5_RF00160_FAOM01456788.1,7.fasta
RF01758_CP007039.1,0_RF02115_CM000666.2,2.fasta
RF02981_FR884369.1,1_RF00677_APLE01009870.1,9.fasta
RF00247_BDDD01000631.1,3_RF03112_URS0000D658B8_12908,6.fasta
RF02115_AYUG01149701.1,2_RF02272_CM000299.1,4.fasta
RF02981_URS0000D67944_12908,1_RF00160_FAOM01456788.1,7.fasta
RF01758_CP007039.1,0_RF00677_AZNC01101182.1,9.fasta
RF03115_URS0000D6A95E_12908,5_RF03112_URS0000D6A2BF_12908,6.fasta
RF00247_CM004281.1,3_RF02272_CM001503.2,4.fasta
RF01758_MUIN01000009.1,0_RF03015_CP017717.1,8.fasta
RF01758_CP001157.1,0_RF00160_CM000851.2,7.fasta
RF01758_MUIN01000009.1,0_RF00160_CM000851.2,7.fasta
RF02272_CM000299.1,4_RF00160_JZJH01051268.1,7.fasta
RF01

RF01758_MKEP01000008.1,0_RF02272_JH835498.1,4.fasta
RF01758_CP001157.1,0_RF02272_AYUG01148198.1,4.fasta
RF00247_BDDD01000631.1,3_RF00247_GL377605.1,3.fasta
RF02272_AFSB01081804.1,4_RF00160_AOTI010592543.1,7.fasta
RF00677_CM002925.1,9_RF00677_KB870811.1,9.fasta
RF01758_MUIN01000009.1,0_RF01758_JXDG01000018.1,0.fasta
RF02981_URS0000D682B8_12908,1_RF03112_URS0000D68791_12908,6.fasta
RF03112_URS0000D658B8_12908,6_RF00677_CM001221.2,9.fasta
RF00160_CM000843.2,7_RF00160_BDDD01000122.1,7.fasta
RF03112_URS0000D6B670_12908,6_RF00677_APLE01009870.1,9.fasta
RF01758_MKEP01000008.1,0_RF03015_URS0000D65F98_12908,8.fasta
RF03115_URS0000D68E9D_104623,5_RF03015_URS0000D6CE4F_12908,8.fasta
RF02272_AFSB01081804.1,4_RF03015_URS0000D6AE16_12908,8.fasta
RF03112_URS0000D6C0C3_12908,6_RF00160_CM000843.2,7.fasta
RF03115_URS0000D69010_671990,5_RF03115_URS0000D6A95E_12908,5.fasta
RF03112_URS0000D6C034_12908,6_RF00677_KB846828.1,9.fasta
RF02272_ALWT01213357.1,4_RF03015_URS0000D687AC_298653,8.fasta
RF03112_URS0000

RF02115_AAGV020057930.1,2_RF00160_ATGF01007502.1,7.fasta
RF03112_URS0000D68791_12908,6_RF03015_URS0000D6CE4F_12908,8.fasta
RF02115_AAPE02007756.1,2_RF03115_URS0000D67FBC_12908,5.fasta
RF02981_URS0000D67944_12908,1_RF00160_CM000851.2,7.fasta
RF02981_URS0000D6CA3C_12908,1_RF00677_CM000834.2,9.fasta
RF02272_AFSB01081804.1,4_RF00677_APLE01009870.1,9.fasta
RF02115_CM000554.1,2_RF00247_FAOM01002169.1,3.fasta
RF01758_MKEP01000008.1,0_RF02272_ALWT01213357.1,4.fasta
RF02981_URS0000D67944_12908,1_RF02272_CM000299.1,4.fasta
RF03115_URS0000D6B412_1162294,5_RF03015_URS0000D687AC_298653,8.fasta
RF02272_JH835498.1,4_RF00160_AOTI010592543.1,7.fasta
RF02981_URS0000D68152_12908,1_RF02115_AYUG01149701.1,2.fasta
RF00677_KB870811.1,9_RF00677_JNGA01013975.1,9.fasta
RF03112_URS0000D68791_12908,6_RF00677_CM000834.2,9.fasta
RF02115_GL010057.1,2_RF00247_CM004281.1,3.fasta
RF02115_AAPE02007756.1,2_RF02272_AFSB01081804.1,4.fasta
RF00160_FAOM01456788.1,7_RF03015_JGYV01000001.1,8.fasta
RF02272_CM001584.2,4_RF03112_

RF03112_URS0000D658B8_12908,6_RF00160_CM000843.2,7.fasta
RF02115_ALWT01147276.1,2_RF00247_CM007371.1,3.fasta
RF02115_CM000554.1,2_RF03015_CP017717.1,8.fasta
RF02272_APMT01059419.1,4_RF03112_URS0000D6B670_12908,6.fasta
RF02115_AYUG01149701.1,2_RF02272_AYUG01148198.1,4.fasta
RF00160_FAOM01456788.1,7_RF00677_KB870811.1,9.fasta
RF00160_BDDD01000122.1,7_RF00677_LFYR01000981.1,9.fasta
RF01758_CP007039.1,0_RF02272_DS562892.1,4.fasta
RF03115_URS0000D6B412_1162294,5_RF00160_AOTI010592543.1,7.fasta
RF02981_URS0000D6CA3C_12908,1_RF03015_URS0000D65A6E_867903,8.fasta
RF01758_MUIN01000009.1,0_RF02115_CM000318.3,2.fasta
RF03112_URS0000D6A2BF_12908,6_RF00160_CM000851.2,7.fasta
RF02115_CM000554.1,2_RF00677_KB870811.1,9.fasta
RF00247_AOTI010773815.1,3_RF02272_CM001503.2,4.fasta
RF01758_AMWJ01000044.1,0_RF02115_CM000554.1,2.fasta
RF03115_URS0000D69010_671990,5_RF00677_ATGF01025917.1,9.fasta
RF02981_URS0000D6CA3C_12908,1_RF03112_URS0000D6C0C3_12908,6.fasta
RF01758_JXDG01000018.1,0_RF03115_URS0000D661E2_12

...

RF02115_ALWT01147276.1,2_RF03015_URS0000D67E59_867903,8.fasta
RF00160_CM009297.1,7_RF00677_CM000834.2,9.fasta
RF02981_URS0000D67A8F_12908,1_RF00677_LFYR01000981.1,9.fasta
RF03015_JGYV01000001.1,8_RF00677_LFYR01000981.1,9.fasta
RF03112_URS0000D658B8_12908,6_RF03015_URS0000D67C15_12908,8.fasta
RF00247_JRRC01446979.1,3_RF02272_DS562892.1,4.fasta
RF02272_CM001503.2,4_RF00160_GL348713.1,7.fasta
RF01758_CP001157.1,0_RF03115_URS0000D67FBC_12908,5.fasta
RF02981_URS0000D67944_12908,1_RF00677_CM001221.2,9.fasta
RF02115_ABRN01149944.1,2_RF00247_GL377605.1,3.fasta
RF03115_URS0000D68239_12908,5_RF03015_URS0000D6D2FA_867903,8.fasta
RF02272_CM001503.2,4_RF03015_URS0000D65A6E_867903,8.fasta
RF00160_CM000843.2,7_RF00160_ATGF01007502.1,7.fasta
RF02981_FR884369.1,1_RF00247_CM007371.1,3.fasta
RF03112_URS0000D6B04D_12908,6_RF03112_URS0000D6C034_12908,6.fasta
RF02981_URS0000D6CA3C_12908,1_RF02272_AYUG01148198.1,4.fasta
RF01758_AJMR01000227.1,0_RF03115_URS0000D67FBC_12908,5.fasta
RF01758_AMWJ01000044.1,0_RF0

RF01758_CP001157.1,0_RF02272_ALWT01213357.1,4.fasta
RF02115_GL010057.1,2_RF03015_URS0000D6AE16_12908,8.fasta
RF01758_MKEP01000008.1,0_RF02272_CM001503.2,4.fasta
RF01758_JXDG01000018.1,0_RF02115_AAPE02007756.1,2.fasta
RF00247_GL377605.1,3_RF02272_APMT01059419.1,4.fasta
RF02115_AAPE02007756.1,2_RF03015_URS0000D6CE4F_12908,8.fasta
RF01758_AMWJ01000044.1,0_RF02115_ALWT01147276.1,2.fasta
RF00160_FAOM01456788.1,7_RF03015_URS0000D65F98_12908,8.fasta
RF02115_ABRN01149944.1,2_RF02115_ALWT01147276.1,2.fasta
RF02981_URS0000D67A8F_12908,1_RF02981_FR884369.1,1.fasta
RF02115_CM000554.1,2_RF00247_JRRC01446979.1,3.fasta
RF02981_URS0000D66007_12908,1_RF00247_CM004281.1,3.fasta
RF02272_ALWT01213357.1,4_RF03112_URS0000D68791_12908,6.fasta
RF02115_CM000554.1,2_RF03112_URS0000D6A2BF_12908,6.fasta
RF02115_CM000318.3,2_RF03115_URS0000D6B412_1162294,5.fasta
RF00247_JRRC01446979.1,3_RF00677_AZNC01101182.1,9.fasta
RF02981_URS0000D66007_12908,1_RF02981_FR884369.1,1.fasta
RF03115_URS0000D68239_12908,5_RF00160_ATG

RF00247_CM003506.1,3_RF00247_JRRC01446979.1,3.fasta
RF02115_AAGV020057930.1,2_RF00247_CM001748.1,3.fasta
RF00677_ATGF01025917.1,9_RF00677_LFYR01000981.1,9.fasta
RF03115_URS0000D67FBC_12908,5_RF03112_URS0000D68791_12908,6.fasta
RF03015_URS0000D6CE4F_12908,8_RF00677_AZNC01101182.1,9.fasta
RF02981_URS0000D69ECC_12908,1_RF03112_URS0000D6A2BF_12908,6.fasta
RF03115_URS0000D661E2_1223570,5_RF03015_URS0000D67C15_12908,8.fasta
RF00160_FAOM01456788.1,7_RF03015_URS0000D687AC_298653,8.fasta
RF01758_AE016853.1,0_RF02272_APMT01059419.1,4.fasta
RF02115_CM000554.1,2_RF02272_AFSB01081804.1,4.fasta
RF03115_URS0000D6B412_1162294,5_RF03115_URS0000D68E9D_104623,5.fasta
RF02981_URS0000D67944_12908,1_RF03112_URS0000D68791_12908,6.fasta
RF03112_URS0000D6A12F_12908,6_RF00160_CM000851.2,7.fasta
RF01758_AMWJ01000044.1,0_RF02272_AYUG01148198.1,4.fasta
RF01758_MKEP01000008.1,0_RF02115_AAPE02007756.1,2.fasta
RF03112_URS0000D6A12F_12908,6_RF00160_CM004278.1,7.fasta
RF02981_URS0000D682B8_12908,1_RF02115_CM000554.1,2.

RF02981_URS0000D6C41F_12908,1_RF03015_URS0000D687AC_298653,8.fasta
RF02272_CM001503.2,4_RF00677_KB846828.1,9.fasta
RF02115_FR853097.2,2_RF03015_URS0000D67C15_12908,8.fasta
RF02981_URS0000D67A8F_12908,1_RF02115_FR853097.2,2.fasta
RF02115_GL010057.1,2_RF02272_CM001955.2,4.fasta
RF01758_AJMR01000227.1,0_RF03115_URS0000D65DBE_198628,5.fasta
RF00247_AOTI010773815.1,3_RF00247_CM003506.1,3.fasta
RF01758_MUIN01000009.1,0_RF03015_JGYV01000001.1,8.fasta
RF02272_DS562892.1,4_RF03015_URS0000D67C15_12908,8.fasta
RF02272_JH835498.1,4_RF02272_CM000299.1,4.fasta
RF00247_FAOM01002169.1,3_RF00677_LFYR01000981.1,9.fasta
RF00247_CM003506.1,3_RF03112_URS0000D658B8_12908,6.fasta
RF02272_DS562892.1,4_RF00677_LFYR01000981.1,9.fasta
RF00247_GL377605.1,3_RF03015_CP017717.1,8.fasta
RF03115_URS0000D69010_671990,5_RF00160_JZJH01051268.1,7.fasta
RF00160_GL348713.1,7_RF00677_JNGA01013975.1,9.fasta
RF02981_URS0000D6C41F_12908,1_RF03115_URS0000D6B412_1162294,5.fasta
RF03115_URS0000D6959E_561230,5_RF00677_KB870811.1,9.

RF00247_JRRC01446979.1,3_RF03115_URS0000D69010_671990,5.fasta
RF01758_LKBM01000998.1,0_RF00160_JZJH01051268.1,7.fasta
RF00247_AOTI010773815.1,3_RF03015_URS0000D6D2FA_867903,8.fasta
RF03115_URS0000D69010_671990,5_RF00160_FAOM01456788.1,7.fasta
RF01758_JXDG01000018.1,0_RF00677_KB846828.1,9.fasta
RF00247_CM007371.1,3_RF03115_URS0000D69010_671990,5.fasta
RF00247_FAOM01002169.1,3_RF03015_URS0000D65A6E_867903,8.fasta
RF03115_URS0000D6B412_1162294,5_RF00677_APLE01009870.1,9.fasta
RF01758_MUIN01000009.1,0_RF00247_JRRC01446979.1,3.fasta
RF02981_URS0000D6CA3C_12908,1_RF03115_URS0000D68239_12908,5.fasta
RF01758_CP001157.1,0_RF00160_ATGF01007502.1,7.fasta
RF00247_CM001748.1,3_RF00677_ATGF01025917.1,9.fasta
RF02981_URS0000D66007_12908,1_RF03112_URS0000D6C034_12908,6.fasta
RF01758_LADX01000047.1,0_RF00677_KB846828.1,9.fasta
RF01758_LADX01000047.1,0_RF03015_URS0000D65F98_12908,8.fasta
RF02981_FR884369.1,1_RF00160_JZJH01051268.1,7.fasta
RF00247_CM004281.1,3_RF02272_CM001584.2,4.fasta
RF00247_CM004281.

In [15]:
make_matrix("RF02272_JH835498.1,4_RF03115_URS0000D65EA6_682634,5.fasta")

193 193
(1, 1200, 10)


array([[[0., 1., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [12]:
mat.shape

(4950, 1200, 10)

In [13]:
np.save('./ncRNApair_datav5.npy', mat)

##### Getting binary vector (2500,)

In [14]:
pre_vec = []
for pair in sp:
   # print((int(pair.split(",")[1][0]), int(pair.split(",")[2][0])))
    check = (int(pair.split(",")[1][0]) == int(pair.split(",")[2][0]))
    if check == True:
        pre_vec.append(1)
    else:
        pre_vec.append(0)
bin_vec = np.array(pre_vec)

In [31]:
# print(*list(bin_vec))

In [16]:
np.save('./ncRNApair_labelv5.npy', bin_vec)

In [24]:
! grep -c ":" famlables.txt

50


In [111]:
! ls

genelabel_sample.txt
RF00112_AXOM01000023.1,2_RF00112_CP001600.2,2.fasta
RF00112_AXOM01000023.1,2_RF00112_CP003244.1,2.fasta
RF00112_AXOM01000023.1,2_RF00112_GG753567.1,2.fasta
RF00112_AXOM01000023.1,2_RF00112_JH417874.1,2.fasta
RF00112_AXOM01000023.1,2_RF00117_AE014075.1,4.fasta
RF00112_AXOM01000023.1,2_RF00117_AP009240.1,4.fasta
RF00112_AXOM01000023.1,2_RF00117_CP000266.1,4.fasta
RF00112_AXOM01000023.1,2_RF00117_CP000468.1,4.fasta
RF00112_AXOM01000023.1,2_RF00117_U00096.3,4.fasta
RF00112_AXOM01000023.1,2_RF00427_ABRO02064586.1,7.fasta
RF00112_AXOM01000023.1,2_RF00427_ADON01048710.1,7.fasta
RF00112_AXOM01000023.1,2_RF00427_JJRS01081504.1,7.fasta
RF00112_AXOM01000023.1,2_RF00427_JMFX02025490.1,7.fasta
RF00112_AXOM01000023.1,2_RF00427_LZPO01087241.1,7.fasta
RF00112_AXOM01000023.1,2_RF01955_ALWS01040030.1,9.fasta
RF00112_AXOM01000023.1,2_RF01955_CM000030.3,9.fasta
RF00112_AXOM01000023.1,2_RF01955_CM001387.3,9.fasta
RF00112_AXOM01000023.1,2_RF01955_GL897050.1,9.fasta
RF00112_AXOM01000023.

##### NEW VARIANT; with more adequate order (in accordance with famlables.txt)

In [10]:
''' 
Here we make a list of ordered file-names 
We got *fam70_test.txt* earlier using Creating_sample.ipynb
'''

new_sp = []
fnames = []
order = []
with open("fam70_test.txt", 'r') as fname:
    for string in fname:
        new_sp.append(string.strip()[:-2])
for file in glob.glob("*.fasta"):
    fnames.append(file)
for i in new_sp:
    for j in fnames:
        if i in j.split(",")[0]:
            order.append(j)

In [11]:
# Checking the length of our list
len(order)

1891

In [12]:
# We shoul copy all files with pairwise alignments into the our working directory
mat = int(0)

for file in order: # glob.glob("*.fasta"):
    if type(mat) == int:
        mat = make_matrix(file)
    else:
        mat = np.concatenate((mat, make_matrix(file)), axis=0)
mat.shape

(1891, 1200, 10)

In [13]:
# Save your matrix into file to use it later for neural network (input)
np.save('./ncRNApair_datav7_test.npy', mat)

In [14]:
# Getting a binary vector (are the sequences belong to the same family)
pre_vec = []
for pair in order:
    check = (int(pair.split(",")[1][0]) == int(pair.split(",")[2][0]))
    if check == True:
        pre_vec.append(0) # old version: pre_vec.append(1)
    else:
        pre_vec.append(1) # old version: pre_vec.append(0)
bin_vec = np.array(pre_vec)

In [15]:
# Save your vector into file to use it later for neural network (input)
np.save('./ncRNApair_labelv7_test.npy', bin_vec)

In [16]:
# print(*list(bin_vec))
len(bin_vec)

1891

In [17]:
mat.shape

(1891, 1200, 10)